In [ ]:
pip install numpy tensorflow matplotlib scikit-learn pillow

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt

tf.get_logger().setLevel("ERROR")

dataset_choice = input("Enter mnist: ")
epochs = int(input("Enter epochs: 30-100: "))
batch_size = int(input("Enter batch size (64/128): "))
noise_dim = int(input("Enter noise dimension (50/100): "))
learning_rate = float(input("Enter learning rate (e.g. 0.0002): "))
save_interval = int(input("Save interval (e.g. 5): "))

IMG_SHAPE = (28,28,1)

os.makedirs("generated_samples", exist_ok=True)
os.makedirs("final_generated_images", exist_ok=True)

(x_train, y_train), (_, _) = tf.keras.datasets.mnist.load_data()

# Normalize to [-1, 1]
x_train = (x_train.astype("float32") - 127.5) / 127.5
x_train = np.expand_dims(x_train, axis=-1)

# Use 40k images
x_train = x_train[:40000]
y_train = y_train[:40000]

dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(len(x_train)).batch(batch_size)


def build_generator():
    return tf.keras.Sequential([
        layers.Dense(7*7*512, input_dim=noise_dim),  # <- Dense 512 applied here
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),
        layers.Reshape((7,7,512)),

        layers.Conv2DTranspose(256,4,strides=2,padding="same"),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        layers.Conv2DTranspose(128,4,strides=2,padding="same"),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        layers.Conv2D(1,3,padding="same",activation="tanh")
    ])

# ================= DISCRIMINATOR =================
def build_discriminator():
    return tf.keras.Sequential([
        layers.Conv2D(64,3,strides=2,padding="same",input_shape=IMG_SHAPE),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),

        layers.Conv2D(128,3,strides=2,padding="same"),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),

        layers.Flatten(),
        layers.Dense(1,activation="sigmoid")
    ])

generator = build_generator()
discriminator = build_discriminator()

# Two optimizers (required by modern Keras)
d_opt = tf.keras.optimizers.Adam(learning_rate, beta_1=0.5)
g_opt = tf.keras.optimizers.Adam(learning_rate, beta_1=0.5)

discriminator.compile(loss="binary_crossentropy", optimizer=d_opt, metrics=["accuracy"])

# Build GAN
z = layers.Input(shape=(noise_dim,))
fake = generator(z)

discriminator.trainable = False
valid = discriminator(fake)

gan = tf.keras.Model(z, valid)
gan.compile(loss="binary_crossentropy", optimizer=g_opt)

discriminator.trainable = True

# ================= SAVE 5x5 GRID =================
def save_images(epoch):
    noise = np.random.normal(0,1,(25,noise_dim))
    imgs = generator.predict(noise, verbose=0)
    imgs = 0.5*imgs + 0.5

    fig,axs = plt.subplots(5,5)
    c=0
    for i in range(5):
        for j in range(5):
            axs[i,j].imshow(imgs[c,:,:,0], cmap="gray")
            axs[i,j].axis("off")
            c+=1
    plt.savefig(f"generated_samples/epoch_{epoch:02d}.png")
    plt.close()

# ================= TRAINING =================
for epoch in range(1, epochs+1):

    for real in dataset:

        batch = real.shape[0]

        noise = np.random.normal(0,1,(batch,noise_dim))
        fake_imgs = generator.predict(noise, verbose=0)

        # Label smoothing
        real_y = np.ones((batch,1))*0.9
        fake_y = np.zeros((batch,1))+0.1

        d_loss_real = discriminator.train_on_batch(real, real_y)
        d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_y)
        d_loss = 0.5*np.add(d_loss_real, d_loss_fake)

        noise = np.random.normal(0,1,(batch,noise_dim))
        g_loss = gan.train_on_batch(noise, np.ones((batch,1))*0.9)

    print(f"Epoch {epoch}/{epochs} | D_loss: {d_loss[0]:.2f} | D_acc: {d_loss[1]*100:.2f}% | G_loss: {g_loss:.2f}")

    if epoch % save_interval == 0:
        save_images(epoch)

# ================= FINAL 100 IMAGES =================
noise = np.random.normal(0,1,(100,noise_dim))
final_imgs = generator.predict(noise, verbose=0)
final_imgs = 0.5*final_imgs + 0.5

for i in range(100):
    plt.imshow(final_imgs[i,:,:,0], cmap="gray")
    plt.axis("off")
    plt.savefig(f"final_generated_images/img_{i}.png")
    plt.close()

# ================= CLASSIFIER (for label distribution) =================
classifier = tf.keras.Sequential([
    layers.Conv2D(32,3,activation="relu",input_shape=IMG_SHAPE),
    layers.MaxPooling2D(),
    layers.Conv2D(64,3,activation="relu"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128,activation="relu"),
    layers.Dense(10,activation="softmax")
])

classifier.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
classifier.fit(x_train, y_train, epochs=3, batch_size=128, verbose=0)

preds = classifier.predict(final_imgs, verbose=0)
pred_labels = np.argmax(preds,axis=1)

u,c = np.unique(pred_labels, return_counts=True)
print("\nLabel Distribution:")
for a,b in zip(u,c):
    print(f"Class {a}: {b}")


Enter mnist: mnist
Enter epochs: 30-100: 30
Enter batch size (64/128): 64
Enter noise dimension (50/100): 100
Enter learning rate (e.g. 0.0002): 0.0001
Save interval (e.g. 5): 1
11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30 | D_loss: 0.43 | D_acc: 0.00% | G_loss: 0.69
Epoch 2/30 | D_loss: 0.50 | D_acc: 0.00% | G_loss: 0.80
Epoch 3/30 | D_loss: 0.61 | D_acc: 0.00% | G_loss: 0.77
Epoch 4/30 | D_loss: 0.69 | D_acc: 0.00% | G_loss: 0.70
Epoch 5/30 | D_loss: 0.81 | D_acc: 0.00% | G_loss: 0.70
Epoch 6/30 | D_loss: 0.89 | D_acc: 0.00% | G_loss: 0.71
Epoch 7/30 | D_loss: 0.93 | D_acc: 0.00% | G_loss: 0.73
Epoch 8/30 | D_loss: 0.95 | D_acc: 0.00% | G_loss: 0.72
Epoch 9/30 | D_loss: 0.97 | D_acc: 0.00% | G_loss: 0.72
Epoch 10/30 | D_loss: 0.98 | D_acc: 0.00% | G_loss: 0.73
Epoch 11/30 | D_loss: 0.97 | D_acc: 0.00% | G_loss: 0.74
Epoch 12/30 | D_loss: 0.96 | D_acc: 0.00% | G_loss: 0.76
Epoch 13/30 | D_loss: 0.97 | D_acc: 0.00% | G_loss: 0.75
Epoch 14/30 | D_loss: 0.99 | D_acc: 0.00% | G_loss: 0.77
Epoch 15/30 | D_loss: 0.98 | D_acc: 0.00% | G_loss: 0.77
Epoch 16/30 | D_loss: 1.00 | D_acc: 0.00% | G_loss: 0.78
Epoch 17/30 | D_loss: 1.01 | D_acc: 0.00% | G_loss: 0.81
Epoch 18/30 | D_loss: 1.01 | D_acc: 0.00

In [ ]:
import zipfile
from google.colab import files

# Zip the generated_samples folder
with zipfile.ZipFile('generated_samples.zip', 'w') as zipf:
    for root, _, filenames in os.walk('generated_samples'):
        for filename in filenames:
            zipf.write(os.path.join(root, filename), os.path.relpath(os.path.join(root, filename), 'generated_samples'))

# Zip the final_generated_images folder
with zipfile.ZipFile('final_generated_images.zip', 'w') as zipf:
    for root, _, filenames in os.walk('final_generated_images'):
        for filename in filenames:
            zipf.write(os.path.join(root, filename), os.path.relpath(os.path.join(root, filename), 'final_generated_images'))

# You can also combine them into a single zip file if preferred
with zipfile.ZipFile('gan_generated_images.zip', 'w') as zipf:
    for folder in ['generated_samples', 'final_generated_images']:
        for root, _, filenames in os.walk(folder):
            for filename in filenames:
                zipf.write(os.path.join(root, filename), os.path.relpath(os.path.join(root, filename), os.path.join(folder, '..')))


print("Generated samples zipped into generated_samples.zip")
print("Final generated images zipped into final_generated_images.zip")
print("All generated images zipped into gan_generated_images.zip")

# To download the combined zip file:
files.download('gan_generated_images.zip')


Generated samples zipped into generated_samples.zip
Final generated images zipped into final_generated_images.zip
All generated images zipped into gan_generated_images.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>